<a href="https://colab.research.google.com/github/flaviorv/pb_ai_ml/blob/main/heart_tp1_pb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, RobustScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Questão 1 - Carregamento de Dados: Identifique as características (features) e o alvo (target)
# nos conjuntos de dados, compreendendo sua estrutura e importância.

#Obtendo dataset do repositório
url = 'https://raw.githubusercontent.com/professortiagoinfnet/inteligencia_artificial/refs/heads/main/heart.csv'
heart_df = pd.read_csv(url).dropna()

# Neste dataset, 'Sex', 'ChestPainType', 'FastingBS', 'RestingECG', 'ExerciseAngina'
# e 'ST_Slope' são features categóricas. ‘Age’, ‘RestingBP’, ‘Cholesterol’, ‘MaxHR’, e
# ‘Oldpeak’, são features contínuas. ‘HeartDisease’ é o target.
# As features são as características de algo que tentamos encontrar padrões para
# treinar a IA através desses padrões. Nesse caso, são dados de um paciente, como
# a idade, colesterol, etc. O target, é o que tentaremos prever.
# Em um aprendizado supervisionado como esse, já temos o target nos dados de aprendizado para
# ajudar no treino e à encontrar um padrão entre as features e o target.
# O target deste dataset é um booleano para cardiopata ou não.

#Separando as features e o target
columns = list(heart_df.head())

target_name = columns.pop()
feature_names = columns

target = heart_df[target_name]
features = heart_df[feature_names]

print(f'Features:\n{features.head()}', end='\n\n')
print(f'Target:\n{target.head()}', end='\n\n')

# Questão 2 - Divisão de Dados: Separe os dados em conjuntos de treino e validação,
# essenciais para o desenvolvimento e avaliação de modelos de ML.
# Você irá dividir 80% dos dados para treino e o restante para validação.

#Separando os dados de treino dos de validação
x_train, x_val, y_train, y_val = train_test_split(features, target, train_size=0.80)

# Questão 3 - Transformação de Dados: Utilize transformadores do Scikit-Learn para ajustar e padronizar as variáveis dos dados.

# Transformando variáveis categóricas com BinaryEncoding e OneHotEncoding,
# enquanto as contínuas tem suas escalas modificadas com StandardScaler.
f_continuous = ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']
f_binary = ['Sex', 'ExerciseAngina']
f_multiclass = ['ChestPainType', 'RestingECG', 'ST_Slope']

preprocessor = ColumnTransformer(
    transformers = [
        ('continuous', RobustScaler(), f_continuous),
        ('binary', OrdinalEncoder(), f_binary),
        ('multiclass', OneHotEncoder(handle_unknown='ignore'), f_multiclass)
    ],
    remainder='passthrough'
)

x_train_transformed = preprocessor.fit_transform(x_train)
x_val_transformed = preprocessor.transform(x_val)

# Questão 4 - Modelagem de K-Nearest Neighbors (KNN): Implemente e ajuste o algoritmo KNN, compreendendo suas nuances e aplicabilidade.
# Questão 5 - Avaliação de Modelos: Avalie a qualidade do modelo desenvolvido usando acurácia
# Questão 6 - Análise de Parâmetros: Explore diferentes valores para o parâmetro K do KNN e analise seu impacto nos resultados obtidos (através da acurácia do modelo para os dados de validação).

# Utilizando um loop para treinamento com KNN com diferentes k vizinhos.
results = pd.DataFrame(columns=['K Neighbors', 'Accuracy'])
for k in range(10, 511, 50):
  # Instância do KNN
  knn = KNeighborsClassifier(n_neighbors=k)
  # Realizando o treinamento
  knn.fit(x_train_transformed, y_train)
  # Prevendo os dados de validação
  predicted = knn.predict(x_val_transformed)
  # Verificando acurácia dos dados previstos
  results.loc[len(results)] = [k, f'{accuracy_score(y_val, predicted):.2f}']
print(f'Results:\n{results}')

Features:
   Age Sex ChestPainType  RestingBP  Cholesterol  FastingBS RestingECG  MaxHR  \
0   40   M           ATA        140          289          0     Normal    172   
1   49   F           NAP        160          180          0     Normal    156   
2   37   M           ATA        130          283          0         ST     98   
3   48   F           ASY        138          214          0     Normal    108   
4   54   M           NAP        150          195          0     Normal    122   

  ExerciseAngina  Oldpeak ST_Slope  
0              N      0.0       Up  
1              N      1.0     Flat  
2              N      0.0       Up  
3              Y      1.5     Flat  
4              N      0.0       Up  

Target:
0    0
1    1
2    0
3    1
4    0
Name: HeartDisease, dtype: int64

Results:
    K Neighbors Accuracy
0            10     0.86
1            60     0.89
2           110     0.89
3           160     0.89
4           210     0.85
5           260     0.86
6           310    